# Select top-N assets with good indicators
The objective is to calculate the standard indicators for a top N number of assets. Then use the indicators to decide on a potentially significant set of assets to consider for the portfolio. Thereafter, apply the MPT monte carlo algorithm to construct a weigted portfolio. 

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore') #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; 
    else set it to False to turn off debugging mode '''
debug = True

## Initialize the classes

In [147]:
import os
import sys
from datetime import datetime, date, timedelta
from pyspark.sql import functions as F

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
from mining.modules.assets.etp import dailyTopN as topN
from mining.modules.assets.etp import performIndex as idx
from utils.modules.ml.timeseries import rollingstats as stat

''' restart initiate classes '''
if debug:
    import importlib
    topN = importlib.reload(topN)
    idx = importlib.reload(idx)
    stat = importlib.reload(stat)

__desc__ = "analyze crypto market capitalization time series data"
clsTopN = topN.WeightedPortfolio(desc=__desc__)
clsIdx = idx.Portfolio(desc=__desc__)
clsStat = stat.RollingStats(desc=__desc__)
print("\nClass initialization and load complete!")

All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional PERFORMINDEX-libraries in ETP-package of ASSETS-module imported successfully!
All packages in utils ml timeseries RollingStats imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
dailyTopN Class initialization complete
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete

Class initialization and load complete!


## Select topN assets

### Read top N mcap assets from tip sql db

Set the following parameters to select the mcap data from the database
* ```_num_assets``` (integer) limits the number of asset count
* ```_mcap_val_lb```(decimal) limits the asset selection by mcap_value
* ```_date``` (datetime) selects assets with values for that day
* ```_table```(string) by default is 'warehouse.mcap_past' where mcap daily data is stored

Extends the ```utils/etl/load/sparkDBwls``` package to read the data from database table


### Construct a dict with selected assets
The dictionary serves as an input to the ```mining/modules/assets/etp/performIndex``` package to compuiting the index values.

[Common metrics for assessing asset strength](https://cointelegraph.com/learn/the-most-common-crypto-metrics-a-beginners-guide)

In [148]:
_num_assets=5
_mcap_val_lb=10000000.0
_date=date(2023,3,28)
_table='warehouse.mcap_past'
# _idx_types=['adx']
# _idx_types=['adx','rsi','mfi','macd']
_idx_types = {
    "MFI":{"DATEATTR":'mcap_date',"VALUEATTR":'mcap_simp_ror',
           "WINLENGTH":14,"WINUNIT":'DAY',"LBOUND":50.0},
    "ADX": {"DATEATTR":'mcap_date',"VALUEATTR":'mcap_simp_ror',
            "WINLENGTH":14,"WINUNIT":'DAY',"LBOUND":50.0},
    "RSI":{"DATEATTR":'price_date',"VALUEATTR":'mcap_simp_ror',
           "WINLENGTH":14,"WINUNIT":'DAY',"LBOUND":50.0},
#     "MACD":{"DATEATTR":'price_date',"VALUEATTR":'price_log_ror',
#             "WINLENGTH":20,"WINUNIT":'DAY',"LBOUND":50.0},
}
kwargs={
    "TABLENAME" :_table,
    "ASSETCOUNT":_num_assets,
    "INDEXTYPELIST":_idx_types,
    "STANDARDIZE" :'minmax',
    "COLPREFIX" :'scaled',
#     "WINLENGTH" :20,
#     "WINUNIT" :'DAY',
    "PCALOWERBOUND":0.7,
}

_portfolio=clsTopN.select_top_assets(
    mcap_date=_date,
    mcap_value_lb=_mcap_val_lb,
    indicators=_idx_types,
    **kwargs,
)

sort_list = [x for x in _idx_types.keys()]
_portfolio[:_num_assets].sort_values(sort_list,ascending=False)

,mcap_past_pk,uuid,asset_name,mcap_date,mcap_value,price_date,price_value,MFI,ADX,RSI
1,44837,64536d2d4a6e0a5e3a66f36b,bnb,2023-03-28,4.976230e+10,2023-03-28,315.159231,60.365588,14.272083,51.479892
3,13583,6453c36f4a6e0a5e3a7d1cf1,shiba_inu,2023-03-28,6.146102e+09,2023-03-28,0.000010,54.380103,11.922092,36.719054
2,46173,6453824d4a6e0a5e3a6c6bdd,dogecoin,2023-03-28,1.015058e+10,2023-03-28,0.073229,54.115075,10.889207,40.550997
0,25717,645387394a6e0a5e3a6dbad8,ethereum,2023-03-28,2.103219e+11,2023-03-28,1748.638230,53.760582,16.009911,52.641604
4,24741,64536da74a6e0a5e3a670f67,bitcoin,2023-03-28,5.262506e+11,2023-03-28,27224.459302,40.133419,14.301318,58.423716


## Write portfolio to mongodb

In [146]:
d = _portfolio['mcap_date'][0]
print(d)
d = str(d).replace(" ","T").split("T")[0]
print(d)

2023-03-28 00:00:00
2023-03-28


In [149]:
kwargs={
    "DESTINDBNAME":'tip-portfolio',
    "COLLPREFIX" : 'cherry_picked',
    "UIDSLIST" : ['uuid'],
    "DATECOLLIST":['asset.mcap.date','asset.price.date']
}
_saved_port = clsTopN.write_asset_picks_to_db(
    portfolio_data=_portfolio,
    cols_dict={},
    **kwargs,
)
_saved_port[:1]

[{'mcap.db.fk': 25717,
  'uuid': '645387394a6e0a5e3a6dbad8',
  'asset.name': 'ethereum',
  'asset.mcap.date': Timestamp('2023-03-28 00:00:00'),
  'asset.mcap.value': 210321902058.243,
  'asset.price.date': Timestamp('2023-03-28 00:00:00'),
  'asset.price.value': 1748.63823000541,
  'index.mfi': 53.760582468694764,
  'index.adx': 16.00991055867257,
  'index.rsi': 52.641604399410255,
  'audit.mod.by': 'NUWAN',
  'audit.mod.dt': '2023-06-07T19:03:48',
  'audit.mod.proc': 'dailyTopN-dailyTopN function <write_asset_picks_to_db>',
  '_id': ObjectId('648064142e45a8d31d9925f2')}]

In [20]:
_saved_port

[{'mcap_past_pk': 25717,
  'uuid': '645387394a6e0a5e3a6dbad8',
  'asset_name': 'ethereum',
  'mcap_date': datetime.date(2023, 3, 28),
  'mcap_value': 210321902058.243,
  'price_date': '2023-03-28T00:00:00',
  'price_value': 1748.63823000541,
  'MFI': 53.760582468694764,
  'ADX': 16.00991055867257,
  'RSI': 52.641604399410255},
 {'mcap_past_pk': 44837,
  'uuid': '64536d2d4a6e0a5e3a66f36b',
  'asset_name': 'bnb',
  'mcap_date': datetime.date(2023, 3, 28),
  'mcap_value': 49762298122.6283,
  'price_date': '2023-03-28T00:00:00',
  'price_value': 315.159230724368,
  'MFI': 60.36558781796708,
  'ADX': 14.272083409995986,
  'RSI': 51.47989206817888},
 {'mcap_past_pk': 46173,
  'uuid': '6453824d4a6e0a5e3a6c6bdd',
  'asset_name': 'dogecoin',
  'mcap_date': datetime.date(2023, 3, 28),
  'mcap_value': 10150577357.4086,
  'price_date': '2023-03-28T00:00:00',
  'price_value': 0.0732292029779081,
  'MFI': 54.1150748038993,
  'ADX': 10.889206526621413,
  'RSI': 40.55099707412268},
 {'mcap_past_pk': 1

In [22]:
''' DEPRECATED already moved to utils/ml/timeriese/rollingstats package'''
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf
# from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import DoubleType, StructField

def exp_ma(df, group_col='asset_name', sort_col='mcap_date',**kwargs):
    schema = (df.select('*')
        .schema.add(StructField('ewma', DoubleType())))

    @pandas_udf(schema, F.PandasUDFType.GROUPED_MAP)
    def ema(pdf):
        pdf['ewma'] = pdf['mcap_value'].ewm(span=5, min_periods=1).mean()
        return pdf

    return df.groupby('asset_name').apply(ema)

# df = spark.createDataFrame(
#     [("a", 1, 1), ("a", 2, 3), ("a", 3, 3), ("b", 1, 10), ("b", 8, 3), ("b", 9, 0)], 
#     ("name", "date", "count")
# )

exp_ma(clsTopN.data).show()

+------------+--------------------+-----------+----------------+------------+----------------+--------+-------------------+--------------------+------------------+------------------+-------------------+--------------------+---------+------------------+------------------+-------------------+--------------------+-------------+--------------------+----------+--------------------+--------------------+-----------+--------------------+-------------+-------------------+
|mcap_past_pk|                uuid|data_source|      asset_name|asset_symbol|    alt_asset_id|currency|         price_date|         price_value|     price_log_ror|    price_simp_ror|          mcap_date|          mcap_value|mcap_rank|      mcap_log_ror|     mcap_simp_ror|        volume_date|         volume_size|volume_change|          created_dt|created_by|        created_proc|         modified_dt|modified_by|       modified_proc|deactivate_dt|               ewma|
+------------+--------------------+-----------+----------------+